In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys,os,re
sys.path.append("../shared/utilsGRF/")
import AnalyseBoundarysearch
from shutil import copyfile
sys.path.append("./scripts")
sys.path.append("./cflinux/")
sys.path.append("./cfmac/")
import BoundaryFinder as BF
import matplotlib as mpl
import PolAB_A_allpars
GRFA=PolAB_A_allpars.interface_GRF_PolAB_A_A

In [ ]:
def return_fullparset(parset,case):
    if case=="any":
        parset2=parset.copy()
    elif case=="difAD_difbnp":
        parset2=np.concatenate((parset[0:12],parset[12:14],parset[12:14],parset[12:14],parset[14:16],parset[14:16],parset[14:16]))
    elif case=="difADsbnp" or case=="difAD_samebnp":
        parset2=np.concatenate((parset[0:12],parset[12:14],parset[12:14],parset[12:14],parset[12:14],parset[12:14],parset[12:14]))
    elif case=="difAD_samebnp_step12":#ni,ia
        parset2=np.concatenate((parset[0:4],parset[0:3],parset[4:5],parset[5:6],parset[1:4],parset[6:8],parset[6:8],parset[6:8],parset[6:8],parset[6:8],parset[6:8]))
    elif case=="difAD_samebnp_step13": #ni,an
        parset2=np.concatenate((parset[0:4],parset[0:3],parset[4:5],parset[0:1],parset[5:6],parset[2:4],parset[6:8],parset[6:8],parset[6:8],parset[6:8],parset[6:8],parset[6:8]))    
    elif case=="difAD_samebnp_step23":#ia,an
        parset2=np.concatenate((parset[0:4],parset[4:5],parset[1:4],parset[0:1],parset[5:6],parset[2:4],parset[6:8],parset[6:8],parset[6:8],parset[6:8],parset[6:8],parset[6:8]))    
    elif case=="difAD_samebnp_step11":
        parset2=np.concatenate((parset[0:4],parset[0:3],parset[4:5],parset[0:3],parset[5:6],parset[6:8],parset[6:8],parset[6:8],parset[6:8],parset[6:8],parset[6:8]))
    elif case=="sameAD_difbp":
        parset2=np.concatenate((parset[0:8],parset[4:8],parset[8:20]))
    elif case=="sameAD_difbp_kuonly":
        kb=parset[8]
        ku1,ku2,ku3,ku4,ku5,ku6=parset[9:]
        bindingar=np.array([kb,ku1,kb,ku2,kb,ku3,kb,ku4,kb,ku5,kb,ku6])
        parset2=np.concatenate((parset[0:8],parset[4:8],bindingar))
    elif case=="sameAD_difbnp":
        parset2=np.concatenate((parset[0:8],parset[4:8],parset[8:10],parset[8:10],parset[8:10],parset[10:12],parset[10:12],parset[10:12]))
    elif case=="empty":
         parset2=np.concatenate((parset[0:8],parset[0:4],parset[8:10],parset[8:10],parset[8:10],parset[8:10],parset[8:10],parset[8:10]))
    else:
        print("unrecognised case, ", case)
        raise ValueError
    return parset2

def get_constraints_npars(case,fcd=0.01,fcu=100):
    if case=="any":
        npars=24
        constraints={4:{'target':0,'fcd':1,'fcu':fcu},5:{'target':1,'fcd':1,'fcu':fcu},6:{'target':2,'fcd':fcd,'fcu':1},7:{'target':3,'fcd':1,'fcu':fcu}, 8:{'target':0,'fcd':1,'fcu':fcu},9:{'target':1,'fcd':1,'fcu':fcu},10:{'target':2,'fcd':fcd,'fcu':1},11:{'target':3,'fcd':1,'fcu':fcu}}
        
    elif case=="difAD_difbnp":
        npars=16
        constraints={4:{'target':0,'fcd':1,'fcu':fcu},5:{'target':1,'fcd':1,'fcu':fcu},6:{'target':2,'fcd':fcd,'fcu':1},7:{'target':3,'fcd':1,'fcu':fcu}, 8:{'target':0,'fcd':1,'fcu':fcu},9:{'target':1,'fcd':1,'fcu':fcu},10:{'target':2,'fcd':fcd,'fcu':1},11:{'target':3,'fcd':1,'fcu':fcu}}
    elif case=="difADsbnp" or case=="difAD_samebnp":
        npars=14
        constraints={4:{'target':0,'fcd':1,'fcu':fcu},5:{'target':1,'fcd':1,'fcu':fcu},6:{'target':2,'fcd':fcd,'fcu':1},7:{'target':3,'fcd':1,'fcu':fcu}, 8:{'target':0,'fcd':1,'fcu':fcu},9:{'target':1,'fcd':1,'fcu':fcu},10:{'target':2,'fcd':fcd,'fcu':1},11:{'target':3,'fcd':1,'fcu':fcu}}
        #parset2=np.concatenate((parset[0:12],parset[12:14],parset[12:14],parset[12:14],parset[12:14],parset[12:14],parset[12:14]))
    elif case=="difAD_samebnp_step12":#ni,ia
        npars=8
        constraints={4:{'target':3,'fcd':1,'fcu':fcu},5:{'target':0,'fcd':1,'fcu':fcu}}
        #parset2=np.concatenate((parset[0:4],parset[0:3],parset[4:5],parset[5:6],parset[1:4],parset[6:8],parset[6:8],parset[6:8],parset[6:8],parset[6:8],parset[6:8]))
    elif case=="difAD_samebnp_step13": #ni,an
        npars=8
        constraints={4:{'target':3,'fcd':1,'fcu':fcu},5:{'target':1,'fcd':1,'fcu':fcu}}
        #parset2=np.concatenate((parset[0:4],parset[0:3],parset[4:5],parset[0:1],parset[5:6],parset[2:4],parset[6:8],parset[6:8],parset[6:8],parset[6:8],parset[6:8],parset[6:8]))
    elif case=="difAD_samebnp_step23":#ia,an
        constraints={4:{'target':0,'fcd':1,'fcu':fcu},5:{'target':1,'fcd':1,'fcu':fcu}}
        npars=8
        #parset2=np.concatenate((parset[0:4],parset[4:5],parset[1:4],parset[0:1],parset[5:6],parset[2:4],parset[6:8],parset[6:8],parset[6:8],parset[6:8],parset[6:8],parset[6:8]))
    elif case=="difAD_samebnp_step11":
        npars=8
        constraints={4:{'target':3,'fcd':1,'fcu':fcu},5:{'target':3,'fcd':1,'fcu':fcu}}
        #parset2=np.concatenate((parset[0:4],parset[0:3],parset[4:5],parset[0:3],parset[5:6],parset[6:8],parset[6:8],parset[6:8],parset[6:8],parset[6:8],parset[6:8]))
    elif case=="sameAD_difbp":
        npars=20
        constraints={4:{'target':0,'fcd':1,'fcu':fcu},5:{'target':1,'fcd':1,'fcu':fcu},6:{'target':2,'fcd':fcd,'fcu':1},7:{'target':3,'fcd':1,'fcu':fcu}}
        #parset2=np.concatenate((parset[0:8],parset[4:8],parset[8:20]))
    elif case=="sameAD_difbp_kuonly":
        #kb=parset[8]
        #ku1,ku2,ku3,ku4,ku5,ku6=parset[9:]
        #bindingar=np.array([kb,ku1,kb,ku2,kb,ku3,kb,ku4,kb,ku5,kb,ku6])
        #parset2=np.concatenate((parset[0:8],parset[4:8],bindingar))
        npars=15
        constraints={4:{'target':0,'fcd':1,'fcu':fcu},5:{'target':1,'fcd':1,'fcu':fcu},6:{'target':2,'fcd':fcd,'fcu':1},7:{'target':3,'fcd':1,'fcu':fcu}}
    elif case=="sameAD_difbnp":
        npars=12
        constraints={4:{'target':0,'fcd':1,'fcu':fcu},5:{'target':1,'fcd':1,'fcu':fcu},6:{'target':2,'fcd':fcd,'fcu':1},7:{'target':3,'fcd':1,'fcu':fcu}}
        #parset2=np.concatenate((parset[0:8],parset[4:8],parset[8:10],parset[8:10],parset[8:10],parset[10:12],parset[10:12],parset[10:12]))
    elif case=="empty":
        npars=10
        constraints={4:{'target':0,'fcd':1,'fcu':fcu},5:{'target':1,'fcd':1,'fcu':fcu},6:{'target':2,'fcd':fcd,'fcu':1},7:{'target':3,'fcd':1,'fcu':fcu}} 
        #parset2=np.concatenate((parset[0:8],parset[0:4],parset[8:10],parset[8:10],parset[8:10],parset[8:10],parset[8:10],parset[8:10]))
    else:
        print("unrecognised case, ", case)
        raise ValueError
    return [constraints,npars]

def compute_syn_2fc(parset,fc1=10,fc2=5,case=None):
    mstars=[]
    parset2=return_fullparset(parset,case)
    
    for i in range(4):
        if i==0:
            A=0.0
            B=0.0
        elif i==1:
            A=2.0
            B=0.0
        elif i==2:
            A=0.0
            B=2.0
        else:
            A=1.0
            B=1.0
        m=GRFA(parset2.copy(),np.array([B]),A) #in some cases I have observed weird behaviour if the array is passed multiple times so it is safest to copy. 
        mstars.append(m)
    if mstars[1]>mstars[2]:
        r=(mstars[1]/mstars[2])
    else:
        r=(mstars[2]/mstars[1])
    if mstars[1]/mstars[0]>fc1 or mstars[2]/mstars[0]>fc1 or r>fc2:
        result=[None,None]
    else:
        if mstars[1]>=mstars[2]:
            result=[np.log2(mstars[3]/mstars[1]),np.log2(mstars[3]/mstars[2])]
        else:#swap A and B
            result=[np.log2(mstars[3]/mstars[2]),np.log2(mstars[3]/mstars[1])]

    return result

In [ ]:
allcases=["any","difAD_difbnp","difAD_samebnp","difAD_samebnp_step12","difAD_samebnp_step13","difAD_samebnp_step23","difAD_samebnp_step11","sameAD_difbp","sameAD_difbp_kuonly","sameAD_difbnp","empty"]
#cases=allcases[0:2]
if True:
    folder="/Users/rosamartinezcorral/Dropbox (HMS)/data/2020_02_synergy/2020_03_15_all_kPolconstraints_1000_cont"
    fullnames=[]
    allouts=dict()
    for case in allcases:
        allouts[case]=dict()
        fldr=folder
        #fldr=os.path.join(folder,case)
        for fc1 in [5,10]:
            for fc2 in [0.5*fc1, 1*fc1]:
                fc1_fc2="fc1=%g_fc2=%g"%(fc1,fc2)
                basename=case+"_fcd=0.001_fcu=1000_"+fc1_fc2
                print(basename)
                out=AnalyseBoundarysearch.plot_boundaries_search(fldr=fldr,getallpoints=True,printtocheck=False,final=True,septime=" ",jid_num="2074010",basename=basename,xlabel='SAB',ylabel='SBA')
                allouts[case][fc1_fc2]=out
                

In [ ]:
import pandas as pd
import re

In [ ]:
fcpat=re.compile("fc1=([0-9\.]+)_fc2=([0-9\.]+)")

# Save dataframe with m0,mA,mB, SAB and SBA

In [ ]:
dfslist=[]
parvalnames=["p"+str(x) for x in range(1,25)]
colnames=["SABo","SBAo", "SAB", "SBA"]+parvalnames+["fc1","fc2","fc1_fc2"]
for key in allouts:
    newdfrows=[]
    print(key)
    for fc in allouts[key]:
        print(fc)
        fc1,fc2=fcpat.findall(fc)[0]
        fc1=float(fc1)
        fc2=float(fc2)
        #if fc2==2:
        #    fc2=2.5 #there was a mistake when saving the number and I used %d instead of %g and so it didn't save as 2.5
        print(fc1,fc2)
        df=allouts[key][fc][1]
        for r in range(len(df)):
            dfrow=df.iloc[r]
            pars=dfrow["parameters"]
            fullparset=return_fullparset(pars,key)
            syn=compute_syn_2fc(pars,fc1=fc1,fc2=fc2,case=key)
            SABo=dfrow["col"]
            SBAo=dfrow["row"]
            SAB,SBA=syn
            if abs(abs(SAB)-abs(SABo))>0.025 or abs(abs(SBA)-abs(SBAo))>0.025:
                print("wrong",SAB,SABo,SBA,SBAo,abs(abs(SAB)-abs(SABo)),abs(abs(SBA)-abs(SBAo)) )
            else:
                row=[SABo,SBAo,SAB,SBA]+list(fullparset)+[fc1,fc2,fc]
                newdfrows.append(row)
    df=pd.DataFrame(np.vstack(newdfrows),columns=colnames)
            

In [ ]:
colnames=['ktia0', 'ktan0', 'ktin0', 'ktni0', 'ktiaA', 'ktanA', 'ktinA', 'ktniA', 'ktiaB', 'ktanB', 'ktinB', 
          'ktniB', 'kbBa', 'kuBa', 'kbBi', 'kuBi','kbBn', 'kuBn', 'kbAa', 'kuAa', 'kbAi', 'kuAi', 'kbAn', 'kuAn',
          'm0', 'mA', 'mB', 'mAB','oldSAB','oldSBA','SAB','SBA','fc1','fc2','fc1fc2']


for knum,case in enumerate(allouts.keys()):
    subdict=allouts[case]
    allrows=[]
    for k2 in subdict.keys():
        fc1,fc2=fcpat.findall(k2)[0]
        fc1=float(fc1)
        fc2=float(fc2)
        if fc2==2:
            fc2=2.5 #there was a mistake when saving the number and I used %d instead of %g and so it didn't save as 2.5

        df2=subdict[k2][1]
        
        pars=df2["parameters"].values
        oldSAB=df2["col"].values
        oldSBA=df2["row"].values

        print(case,len(pars))
        #if False:
        
        for p in range(len(pars)):
            SABo=oldSAB[p]
            SBAo=oldSBA[p]

            row=[]
            parset=pars[p]
            parset2=return_fullparset(parset,case)
            if len(parset2)!=24:
                print("No 24 parameters!")
                sys.exit()
            mstars=[]
            for i in range(4):
                if i==0:
                    A=0.0
                    B=0.0
                elif i==1:
                    A=2.0
                    B=0.0
                elif i==2:
                    A=0.0
                    B=2.0
                else:
                    A=1.0
                    B=1.0
                m=PolAB_A_allpars.interface_GRF_PolAB_A_A(parset2.copy(),np.array([B]),A)
                mstars.append(m)
            if mstars[1]<mstars[2]:
                parset2=np.concatenate((parset2[0:4],parset2[8:12],parset2[4:8],parset2[18:24],parset2[12:18]))
                mstars=[mstars[0],mstars[2],mstars[1],mstars[3]]

            SABn=np.log2(mstars[3]/mstars[1])
            SBAn=np.log2(mstars[3]/mstars[2])
            if (SABo>0 and SABn<0) or (SABo<0 and SABn>0) or (SBAo>0 and SBAn<0) or (SBAo<0 and SBAn>0):
                print("SABs signs not coincident!!!", SABo,SABn,SBAo,SBAn)
            elif ((np.abs(np.abs(SABo)-np.abs(SABn))>0.03) or (np.abs(np.abs(SBAo)-np.abs(SBAn))>0.03)):
                print("SAB or SBA too different", SABo, SABn, SBAo, SBAn)
            else:
                #make A the one with larger expression. so B becomes A, and A becomes B. swap parameters
                
                row.extend(list(parset2))
                row.extend(mstars)
                row.extend([SABo,SBAo,SABn,SBAn,fc1,fc2,k2])
                allrows.append(row)

    df2=pd.DataFrame(np.vstack(allrows),columns=colnames)
    for col in colnames[:-1]:
        df2[col]=pd.to_numeric(df2[col])

    #df2['SAB']=np.log2(df2["mAB"].values/df2["mA"].values)
    #df2['SBA']=np.log2(df2["mAB"].values/df2["mB"].values)
    array=np.ones(len(df2))*(-1)
    SAB=df2['SAB']
    SBA=df2['SBA']
    m1=((SAB>0) & (SBA>0))
    m2=((SAB<0) & (SBA>0))
    m3=((SAB<0) & (SBA<0))
    array[m1]=1
    array[m2]=2
    array[m3]=3
    #array[m4]=4
    df2['quadrant']=array
    df2.name=case
    df2['mA/mB']=df2['mA']/df2['mB']
    #alldf2.append(df2)
    pd.to_pickle(df2,"2020_03_28_kPolconstr1000_cont_%s.df"%case)
    #print(df2.columns)
    #print(df2.iloc[pidxs[0]])
    #print(df2.iloc[pidxs[1]])
    #print(df2.iloc[pidxs[2]])
        
    
    